In [1]:
import numpy as np
import cv2
import os

In [12]:
def distance(x1,x2):
    return np.sqrt(sum((x1-x2)**2))

def knn(train,test,k=5):
    
    dist = []
    m =train.shape[0]
    
    for i in range(m):
        ix = train[i, :-1]
        iy = train[i, -1]
        
        d = distance(ix,test)
        dist.append([d,iy])
        
    
    dk = sorted(dist, key = lambda x: x[0])[:k]
    
    #Get labels
    labels = np.array(dk)[:,-1]
    
    #Get frequencies
    output = np.unique(labels, return_counts= True)
    
    #get index
    index = np.argmax(output[1])
    
    return output[0][index]

In [13]:
import cv2

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

skip = 0

dataset_path = './data/'

face_data = []
labels = []

class_id = 0 #label
names = {} 


#Data Prep
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]
        
        print("Loaded with "+fx)
        
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        
        
        #Create labels
        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)
        
    
# print(face_data)
# print(labels)

face_dataset = np.concatenate(face_data,axis = 0)
face_labels = np.concatenate(labels, axis = 0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)

train_set = np.concatenate((face_dataset,face_labels),axis = 1)

train_set.shape


Loaded with Golu.npy
(11, 30000)
(11, 1)


(11, 30001)

In [14]:
#Testing

while True:
    ret,frame = cap.read()
    
    if ret == False:
        continue
        
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    
    for x,y,w,h in faces:
        offset = 10
        
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))
        
        
        #predicted
        out = knn(train_set,face_section.flatten())
        
        #display name and rectangle around it
        
        pred_name = names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
    
    cv2.imshow("Faces",frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()